# Calculating SED properties

Should SED fitting have already been run on your catalogue of interest, and the best-fitting SED templates saved, you can calculate properties using child classes of `SED_Property_Calculator`.  To do things a little differently, we will load in our test dataset, for which we require a different config file. The imports thus differ from previous notebooks here in that we must first set an environment variable before running galfind.

In [6]:
import astropy.units as u
import os
# os.environ["GALFIND_CONFIG_DIR"] = "/nvme/scratch/work/austind/GALFIND/testing"
# os.environ["GALFIND_CONFIG_NAME"] = "test_galfind_config.ini"

%load_ext autoreload
%autoreload 2
from galfind import Data, Catalogue, EAZY, Bagpipes, Band_SNR_Selector, MUV_Calculator
from galfind.Data import morgan_version_to_dir
from galfind.Property_calculator import MUV_SED_Property_Calculator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We will first load in the JOF catalogue here as usual, including the results of EAZY SED fitting with the fsps_larson template set with no upper limit on redshift.

In [7]:
# Load in a catalogue object
survey = "PRIMER-UDS"
version = "v12"
instrument_names = ["ACS_WFC", "NIRCam"]
forced_phot_band = ["F277W", "F356W", "F444W"]
# survey = "test"
# version = "v0"
# instrument_names = ["ACS_WFC", "NIRCam"]
# forced_phot_band = ["F200W", "F444W"]
aper_diams = [0.32] * u.arcsec
min_flux_pc_err = 10.0

cat = Catalogue.pipeline(
    survey, 
    version, 
    instrument_names = instrument_names, 
    aper_diams = aper_diams,
    forced_phot_band = forced_phot_band,
    min_flux_pc_err = min_flux_pc_err,
    #im_str = ["test"],
    version_to_dir_dict = morgan_version_to_dir,
)

INFO:galfind:Loading galfind created rms_err for F606W
INFO:galfind:Loading galfind created rms_err for F814W
INFO:galfind:Loading galfind created wht for F090W
INFO:galfind:Loading galfind created wht for F115W
INFO:galfind:Loading galfind created wht for F150W
INFO:galfind:Loading galfind created wht for F200W
INFO:galfind:Loading galfind created wht for F277W
INFO:galfind:Loading galfind created wht for F356W
INFO:galfind:Loading galfind created wht for F410M
INFO:galfind:Loading galfind created wht for F444W
INFO:galfind:Loaded aper_diams=<Quantity [0.32] arcsec> for F277W+F356W+F444W
INFO:galfind:Combined mask for Stacked_Band_Data(NIRCam/F277W+F356W+F444W) already exists at /raid/scratch/work/austind/GALFIND_WORK/Masks/PRIMER-UDS/combined/PRIMER-UDS_v12_F277W+F356W+F444W_auto.fits
Calculating depths:   0%|          | 0/11 [00:00<?, ?it/s]
INFO:galfind:Calculated/loaded depths for PRIMER-UDS v12 ACS_WFC+NIRCam
INFO:galfind:Local depth columns already exist in /raid/scratch/work/au

In [8]:
# load EAZY results into catalogue
EAZY_SED_fitter = EAZY({"templates": "fsps_larson", "lowz_zmax": None})
EAZY_SED_fitter(cat, aper_diams[0], update = True)

INFO:galfind:Making .in file for EAZY_fsps_larson_zfree SED fitting for PRIMER-UDS v12 ACS_WFC+NIRCam
INFO:galfind:Made .in file for EAZY_fsps_larson_zfree SED fitting for PRIMER-UDS v12 ACS_WFC+NIRCam. 


Running SED fitting took 0.4s


INFO:galfind:Loading EAZY_fsps_larson property PDFs into PRIMER-UDS v12 ACS_WFC+NIRCam


Loading properties and associated errors took 18.3s


Constructing redshift PDFs: 100%|██████████| 294375/294375 [00:03<00:00, 80555.82it/s]
INFO:galfind:Finished loading EAZY_fsps_larson property PDFs into PRIMER-UDS v12 ACS_WFC+NIRCam
INFO:galfind:Loading EAZY_fsps_larson SEDs into PRIMER-UDS v12 ACS_WFC+NIRCam
Constructing SEDs: 100%|██████████| 294375/294375 [00:15<00:00, 19523.68it/s]
INFO:galfind:Finished loading EAZY_fsps_larson SEDs into PRIMER-UDS v12 ACS_WFC+NIRCam
INFO:galfind:Updating SED results in galfind catalogue object
Updating galaxy SED results: 100%|██████████| 294375/294375 [00:01<00:00, 195344.91it/s]


[SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 SED_result(EAZY_fsps_larson_zfree),
 

In [9]:
# construct MUV calculator
Muv_calculator_eazy = MUV_SED_Property_Calculator(aper_diams[0], EAZY_SED_fitter)
print(Muv_calculator_eazy)
Muv_calculator_eazy(cat, update = True)

INFO:galfind:Loading cat_colname='FLUX_AUTO' from /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v12/ACS_WFC+NIRCam/PRIMER-UDS/(0.32)as/PRIMER-UDS_MASTER_Sel-F277W+F356W+F444W_v12.fits saved as sex_FLUX_AUTO!


INFO:galfind:Loaded FLUX_AUTO from /raid/scratch/work/austind/GALFIND_WORK/Catalogues/v12/ACS_WFC+NIRCam/PRIMER-UDS/(0.32)as/PRIMER-UDS_MASTER_Sel-F277W+F356W+F444W_v12.fits saved as sex_FLUX_AUTO for cat_band_properties[0].keys()=dict_keys(['F606W', 'F814W', 'F090W', 'F115W', 'F150W', 'F200W', 'F277W', 'F356W', 'F410M', 'F444W'])
INFO:galfind:Loading SExtractor extended source corrections for PRIMER-UDS_MASTER_Sel-F277W+F356W+F444W_v12.fits!
Calculating <galfind.Property_calculator.MUV_SED_Property_Calculator object at 0x7f6026ed0320> for CATALOGUE(PRIMER-UDS,v12,ACS_WFC+NIRCam):   0%|          | 0/294375 [00:00<?, ?it/s]/nvme/scratch/software/anaconda3/envs/more_and_more_galfind/lib/python3.12/site-packages/astropy/units/equivalencies.py:249: RuntimeWarning: invalid value encountered in divide
  return x / (wav.to_value(si.AA, spectral()) ** 2 / c_Aps)
/nvme/scratch/work/austind/GALFIND/galfind/SED.py:182: RuntimeWarning: invalid value encountered in scalar divide
  return numerator 

[-20.487081326698185,
 -21.06978844371592,
 -15.87179975321711,
 -19.51888385537963,
 -20.505645358423283,
 -17.571194393361477,
 -18.097152824516026,
 -20.706077493658636,
 -20.87337353526705,
 nan,
 -18.563075604408645,
 -19.658105783591914,
 -17.09666476287606,
 -27.36501659093448,
 -16.251695443849577,
 -18.378040368984657,
 -24.460668966316824,
 -24.52208548885378,
 -24.328023750753893,
 -24.210506376184657,
 -24.21729229578762,
 -24.147563899120172,
 -24.208142103454442,
 -17.31239383687333,
 -20.314298208711755,
 -16.909981518514748,
 -20.005435137049233,
 -17.676077183601624,
 -21.312605814130585,
 -20.8014006148521,
 -20.25796496404446,
 -20.500950219069452,
 -17.711409031935126,
 -16.82244597083697,
 -18.733027046094957,
 -16.03112012652089,
 -12.780100368359228,
 -17.112295048917137,
 -17.373326459719383,
 nan,
 -21.584235657388902,
 -18.075562137082116,
 -21.06319853611088,
 -20.845007262377642,
 -21.108943078076788,
 -21.27586619036755,
 -18.23687561097934,
 -21.5466778854

By default this will run with extended source corrections applied, which can be turned off simply by setting `ext_src_corrs = None` when instantiating the property calculator. A straightforward example of this is shown below...

In [10]:
Muv_calculator_eazy_no_extsrc = MUV_SED_Property_Calculator(aper_diams[0], EAZY_SED_fitter, ext_src_corrs = None)
print(Muv_calculator_eazy_no_extsrc)
Muv_calculator_eazy_no_extsrc(cat, update = True)

Calculating <galfind.Property_calculator.MUV_SED_Property_Calculator object at 0x7f601fe63860> for CATALOGUE(PRIMER-UDS,v12,ACS_WFC+NIRCam):   0%|          | 0/294375 [00:00<?, ?it/s]

/nvme/scratch/software/anaconda3/envs/more_and_more_galfind/lib/python3.12/site-packages/astropy/units/equivalencies.py:249: RuntimeWarning: invalid value encountered in divide
  return x / (wav.to_value(si.AA, spectral()) ** 2 / c_Aps)
/nvme/scratch/work/austind/GALFIND/galfind/SED.py:182: RuntimeWarning: invalid value encountered in scalar divide
  return numerator / denominator
/nvme/scratch/work/austind/GALFIND/galfind/SED.py:528: RuntimeWarning: divide by zero encountered in log10
  MUV = mUV.value - 5 * np.log10(dL / 10.0) + 2.5 * np.log10(1.0 + self.z)
Calculating <galfind.Property_calculator.MUV_SED_Property_Calculator object at 0x7f601fe63860> for CATALOGUE(PRIMER-UDS,v12,ACS_WFC+NIRCam):   3%|▎         | 10298/294375 [00:41<19:35, 241.57it/s]/nvme/scratch/software/anaconda3/envs/more_and_more_galfind/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: divide by zero encountered in log10
  return dex.to(self._function_unit, np.log10(x))
Calcu

[-17.987081326698185,
 -18.637723000948093,
 -14.48557893496103,
 -17.571207666436212,
 -19.528851080513466,
 -15.36658321363166,
 -16.83974819678576,
 -19.038735430749114,
 -19.863630319352577,
 nan,
 -17.682538448082084,
 -19.658105783591914,
 -16.387881290326426,
 -27.017507826679264,
 -15.11245908039302,
 -18.378040368984657,
 -24.460668966316824,
 -24.52208548885378,
 -24.328023750753893,
 -24.210506376184657,
 -24.21729229578762,
 -24.147563899120172,
 -24.208142103454442,
 -17.130827263932996,
 -20.314298208711755,
 -16.909981518514748,
 -20.005435137049233,
 -16.441993256120792,
 -20.859985014651908,
 -20.8014006148521,
 -20.25796496404446,
 -20.500950219069452,
 -17.711409031935126,
 -16.376549211340695,
 -17.340135241173304,
 -16.03112012652089,
 -12.263551561089326,
 -17.112295048917137,
 -16.6249009662034,
 nan,
 -21.151781658755628,
 -17.657967276319997,
 -21.06319853611088,
 -20.845007262377642,
 -21.108943078076788,
 -21.27586619036755,
 -18.23687561097934,
 -21.54667788